# Deploying TAS4K8s to TKG standalone on vSphere 6.7U3

* TKG 1.1
* No load balancer
* Uses vsphere CNS through CSI

In [1]:
#env
export TAS_ROOT=~/Downloads/tas-k8s-beta-workshop-tkg/tanzu-application-service
export TAS_CONFIG_DIR=~/Downloads/tas-k8s-beta-workshop-tkg/configuration-values
export TAS_ARCHIVE=~/Downloads/tanzu-application-service.0.1.0-build.252.tar
mkdir -p $TAS_ROOT $TAS_CONFIG_DIR
cd $TAS_ROOT

In [2]:
#target the right cluster
kubectl config get-contexts

CURRENT   NAME                                                                    CLUSTER                           AUTHINFO                                NAMESPACE
          minikube                                                                minikube                          minikube                                
*         tkg-guest-admin@tkg-guest                                               tkg-guest                         tkg-guest-admin                         
          tkg-mgmt-vsphere-20200422151222-admin@tkg-mgmt-vsphere-20200422151222   tkg-mgmt-vsphere-20200422151222   tkg-mgmt-vsphere-20200422151222-admin   


In [ ]:
kubectl config use-context tkg-guest-admin@tkg-guest

In [ ]:
kubectl get all -o wide --all-namespaces

# Deploy TAS

In [ ]:
#extract TAS downloaded archive
tar -C $TAS_ROOT -xvf $TAS_ARCHIVE 

In [ ]:
#remove resource requirements
curl https://raw.githubusercontent.com/cloudfoundry/cf-for-k8s/master/config-optional/remove-resource-requirements.yml > $TAS_ROOT/custom-overlays/remove-resource-requirements.yml

## Generate config values

In [ ]:
./bin/generate-values.sh -d "tas-tkg.lan" > ../configuration-values/deployment-values.yml

Add entries for Pivnet registry and your own image registry (place at end of file, or in two separate files, system-registry-values.yml and app-registry-values.yml):
```
system_registry:
  hostname: registry.pivotal.io
  username: pivnet-username
  password: password

app_registry:
  hostname: https://index.docker.io/v1/
  repository: "username"
  username: "uername"
  password: "password"
```

In [ ]:
#Copy my secrets from pre-existing location
cp ~/Documents/app-registry-values.yml $TAS_CONFIG_DIR
cp ~/Documents/system-registry-values.yml $TAS_CONFIG_DIR

## Default storage class

In [4]:
kubectl get storageclass

NAME             PROVISIONER              RECLAIMPOLICY   VOLUMEBINDINGMODE   ALLOWVOLUMEEXPANSION   AGE
thin (default)   csi.vsphere.vmware.com   Delete          Immediate           false                  2d15h


TKG uses CNS through the CSI provisioner. Define your default storage class in a new file config-optional/vsphere-csi-storage-class.yaml:

```
---
kind: StorageClass
apiVersion: storage.k8s.io/v1
metadata:
  name: thin
  annotations:
    storageclass.kubernetes.io/is-default-class: "true"
provisioner: csi.vsphere.vmware.com
parameters:
  storagepolicyname: "k8s-storage-policy"
  fstype: ext4
```

Perform additional steps in vcenter:
* Create new category under "tags and custom attributes" assigned to object types Folder, Datastore, Datastore Cluster. Name it "k8s-storage"
* Pick your datastore -> summary -> Tags, Assign...
* Add tag: k8s-storage-mydatastore, pick k8s-storage from category dropdown
* Create a vm storage policy named "k8s-storage-policy", enable tag based placement rules, add tag rule for k8s-storage category, use storage tagged with, browse tags and select k8s-storage-mydatastore. The desired datastore should appear as compatible.


In [ ]:
cp ~/Documents/vsphere-csi-storage-class.yaml config-optional/

In [3]:
kapp deploy -a default-storage-class -f config-optional/vsphere-csi-storage-class.yaml -y

Target cluster 'https://10.10.3.159:6443' (nodes: tkg-guest-control-plane-7xnn9, 3+)

Changes

Namespace  Name  Kind  Conds.  Age  Op  Wait to  Rs  Ri  

Op:      0 create, 0 delete, 0 update, 0 noop
Wait to: 0 reconcile, 0 delete, 0 noop

Succeeded


In [5]:
#deploy tas
./bin/install-tas.sh ../configuration-values

Target cluster 'https://10.10.3.159:6443' (nodes: tkg-guest-control-plane-7xnn9, 3+)
resolve | final: cfidentity/uaa@sha256:016363eba9544ada12c16ac1e66a7e02241b1ec8074d61f5fa8fd6d7edee3d69 -> registry.pivotal.io/tas-for-kubernetes/uaa-annotated@sha256:b2d8c01426f451fa39bc302e3516bf4b5ab2652874f949be4ced26aa451adab7
resolve | final: cloudfoundry/capi-kpack-watcher:956150dae0a95dcdf3c1f29c23c3bf11db90f7a0@sha256:67125e0d3a4026a23342d80e09aad9284c08ab4f7b3d9a993ae66e403d5d0796 -> registry.pivotal.io/tas-for-kubernetes/capi-kpack-watcher-annotated@sha256:8958b568978de3f093571439d93fb4895ccdd281123b79f4eba56c6be121eadf
resolve | final: cloudfoundry/capi:nginx@sha256:51e4e48c457d5cb922cf0f569e145054e557e214afa78fb2b312a39bb2f938b6 -> registry.pivotal.io/tas-for-kubernetes/capi-annotated@sha256:7560ed0a1175f8a93dcb2ee0a26dbe7127fb0410b801b62861491a0f96b07255
resolve | final: cloudfoundry/cloud-controller-ng:33f461df533c7174241b00759bb7622ea37c58c7@sha256:0bc1b2b3e0c2fcfbd76d7c4a311b728a240b92

^                     build-service                                                   Namespace                     -       -    create  reconcile  -   -  
^                     build-service-webhook                                           MutatingWebhookConfiguration  -       -    create  reconcile  -   -  
^                     builders.build.pivotal.io                                       CustomResourceDefinition      -       -    create  reconcile  -   -  
^                     builds.build.pivotal.io                                         CustomResourceDefinition      -       -    create  reconcile  -   -  
^                     cc-api-service-account-superuser                                ClusterRoleBinding            -       -    create  reconcile  -   -  
^                     cf-blobstore                                                    Namespace                     -       -    create  reconcile  -   -  
^                     cf-db                                     

^                     kpack-controller-admin                                          ClusterRole                   -       -    create  reconcile  -   -  
^                     kpack-controller-admin-binding                                  ClusterRoleBinding            -       -    create  reconcile  -   -  
^                     kpack-watcher                                                   ClusterRole                   -       -    create  reconcile  -   -  
^                     kpack-watcher-binding                                           ClusterRoleBinding            -       -    create  reconcile  -   -  
^                     kpack-webhook-certs-mutatingwebhookconfiguration-admin-binding  ClusterRoleBinding            -       -    create  reconcile  -   -  
^                     kpack-webhook-mutatingwebhookconfiguration-admin                ClusterRole                   -       -    create  reconcile  -   -  
^                     meshpolicies.authentication.istio.io      

^                     default                                                         ServiceAccount                -       -    create  reconcile  -   -  
^                     system-registry-credentials                                     Secret                        -       -    create  reconcile  -   -  
cf-system             capi                                                            Service                       -       -    create  reconcile  -   -  
^                     capi-api-server                                                 Deployment                    -       -    create  reconcile  -   -  
^                     capi-clock                                                      Deployment                    -       -    create  reconcile  -   -  
^                     capi-deployment-updater                                         Deployment                    -       -    create  reconcile  -   -  
^                     capi-external-virtual-service             

^                     app-registry-credentials                                        Secret                        -       -    create  reconcile  -   -  
^                     cf-workloads-app-role                                           Role                          -       -    create  reconcile  -   -  
^                     cf-workloads-app-rolebinding                                    RoleBinding                   -       -    create  reconcile  -   -  
^                     cf-workloads-privileged-app-role                                Role                          -       -    create  reconcile  -   -  
^                     cf-workloads-privileged-app-rolebinding                         RoleBinding                   -       -    create  reconcile  -   -  
^                     default                                                         ServiceAccount                -       -    create  reconcile  -   -  
^                     default                                   

...

9:11:50AM: ok: reconcile secret/app-registry-credentials (v1) namespace: cf-workloads
9:11:50AM: ok: reconcile secret/istio-ingressgateway-certs (v1) namespace: istio-system
9:11:51AM: ok: reconcile gateway/istio-ingressgateway (networking.istio.io/v1alpha3) namespace: cf-system
9:11:52AM: ok: reconcile service/kpack-webhook (v1) namespace: kpack
9:11:53AM: ok: reconcile deployment/kpack-webhook (apps/v1) namespace: kpack
9:11:53AM: ok: reconcile role/kpack-webhook-certs-admin (rbac.authorization.k8s.io/v1) namespace: kpack
9:11:53AM: ok: reconcile rolebinding/kpack-webhook-certs-admin-binding (rbac.authorization.k8s.io/v1) namespace: kpack
9:11:53AM: ok: reconcile policy/cf-blobstore-allow-plaintext (authentication.istio.io/v1alpha1) namespace: cf-blobstore
9:11:53AM: ok: reconcile clusterrole/pod-namespace-read (rbac.authorization.k8s.io/v1) cluster
9:11:54AM: ok: reconcile configmap/fluentd-config (v1) namespace: cf-system
9:11:54AM: ok: reconcile service/log-cache (v1) namespace: c

9:12:11AM: ok: reconcile instance/tcpbytereceived (config.istio.io/v1alpha2) namespace: istio-system
9:12:11AM: ok: reconcile instance/tcpconnectionsopened (config.istio.io/v1alpha2) namespace: istio-system
9:12:12AM: ok: reconcile instance/tcpconnectionsclosed (config.istio.io/v1alpha2) namespace: istio-system
9:12:12AM: ok: reconcile handler/prometheus (config.istio.io/v1alpha2) namespace: istio-system
9:12:12AM: ok: reconcile rule/promhttp (config.istio.io/v1alpha2) namespace: istio-system
9:12:12AM: ok: reconcile rule/promtcp (config.istio.io/v1alpha2) namespace: istio-system
9:12:13AM: ok: reconcile rule/promtcpconnectionopen (config.istio.io/v1alpha2) namespace: istio-system
9:12:13AM: ok: reconcile rule/promtcpconnectionclosed (config.istio.io/v1alpha2) namespace: istio-system
9:12:13AM: ok: reconcile handler/kubernetesenv (config.istio.io/v1alpha2) namespace: istio-system
9:12:13AM: ok: reconcile rule/kubeattrgenrulerule (config.istio.io/v1alpha2) namespace: istio-system
9:12:1

9:12:33AM:  ^ Waiting for 1 unavailable replicas
9:12:33AM:  L ok: waiting on replicaset/capi-clock-55479c654f (apps/v1) namespace: cf-system
9:12:33AM:  L ongoing: waiting on pod/capi-clock-55479c654f-rs8w5 (v1) namespace: cf-system
9:12:33AM:     ^ Pending: PodInitializing
9:12:33AM: ongoing: reconcile deployment/capi-deployment-updater (apps/v1) namespace: cf-system
9:12:33AM:  ^ Waiting for 1 unavailable replicas
9:12:33AM:  L ok: waiting on replicaset/capi-deployment-updater-85897b5d5d (apps/v1) namespace: cf-system
9:12:33AM:  L ongoing: waiting on pod/capi-deployment-updater-85897b5d5d-6k2z9 (v1) namespace: cf-system
9:12:33AM:     ^ Condition Ready is not True (False)
9:12:34AM: ongoing: reconcile daemonset/fluentd (apps/v1) namespace: cf-system
9:12:34AM:  ^ Waiting for 4 unavailable pods
9:12:34AM:  L ongoing: waiting on pod/fluentd-jnqpd (v1) namespace: cf-system
9:12:34AM:     ^ Condition Ready is not True (False)
9:12:34AM:  L ongoing: waiting on pod/fluentd-hjq92 (v1) nam

9:13:13AM:     ^ Condition Ready is not True (False)
9:13:17AM: ok: reconcile deployment/cfroutesync (apps/v1) namespace: cf-system
9:13:19AM: ok: reconcile deployment/uaa (apps/v1) namespace: cf-system
9:13:20AM: ongoing: reconcile daemonset/istio-ingressgateway (apps/v1) namespace: istio-system
9:13:20AM:  ^ Waiting for 2 unavailable pods
9:13:20AM:  L ongoing: waiting on pod/istio-ingressgateway-vp2qq (v1) namespace: istio-system
9:13:20AM:     ^ Condition Ready is not True (False)
9:13:20AM:  L ok: waiting on pod/istio-ingressgateway-l2j97 (v1) namespace: istio-system
9:13:20AM:  L ongoing: waiting on pod/istio-ingressgateway-92xqn (v1) namespace: istio-system
9:13:20AM:     ^ Condition Ready is not True (False)
9:13:20AM:  L ok: waiting on controllerrevision/istio-ingressgateway-5799c46547 (apps/v1) namespace: istio-system
9:13:20AM: ---- waiting on 10 changes [251/261 done] ----
9:13:20AM: ok: reconcile deployment/capi-kpack-watcher (apps/v1) namespace: cf-system
9:13:24AM: ongoi

9:14:05AM: ok: reconcile deployment/cf-blobstore-minio (apps/v1) namespace: cf-blobstore
9:14:05AM: ---- waiting on 2 changes [259/261 done] ----
9:14:06AM: ongoing: reconcile daemonset/fluentd (apps/v1) namespace: cf-system
9:14:06AM:  ^ Waiting for 2 unavailable pods
9:14:06AM:  L ongoing: waiting on pod/fluentd-jnqpd (v1) namespace: cf-system
9:14:06AM:     ^ Condition Ready is not True (False)
9:14:06AM:  L ok: waiting on pod/fluentd-hjq92 (v1) namespace: cf-system
9:14:06AM:  L ok: waiting on pod/fluentd-h2fd2 (v1) namespace: cf-system
9:14:06AM:  L ongoing: waiting on pod/fluentd-4qkwz (v1) namespace: cf-system
9:14:06AM:     ^ Condition Ready is not True (False)
9:14:06AM:  L ok: waiting on controllerrevision/fluentd-5446d56677 (apps/v1) namespace: cf-system
9:14:07AM: ongoing: reconcile daemonset/fluentd (apps/v1) namespace: cf-system
9:14:07AM:  ^ Waiting for 1 unavailable pods
9:14:07AM:  L ok: waiting on pod/fluentd-jnqpd (v1) namespace: cf-system
9:14:07AM:  L ok: waiting o

### Configure DNS to external IPs of worker nodes

In [6]:
kubectl get nodes --output='wide'

NAME                              STATUS   ROLES    AGE     VERSION            INTERNAL-IP   EXTERNAL-IP   OS-IMAGE                 KERNEL-VERSION   CONTAINER-RUNTIME
tkg-guest-control-plane-7xnn9     Ready    master   2d17h   v1.18.2+vmware.1   10.10.3.166   10.10.3.166   VMware Photon OS/Linux   4.19.115-3.ph3   containerd://1.3.4
tkg-guest-md-0-7b9ccb8bd7-p874v   Ready    <none>   2d17h   v1.18.2+vmware.1   10.10.3.169   10.10.3.169   VMware Photon OS/Linux   4.19.115-3.ph3   containerd://1.3.4
tkg-guest-md-0-7b9ccb8bd7-qs8ds   Ready    <none>   2d17h   v1.18.2+vmware.1   10.10.3.167   10.10.3.167   VMware Photon OS/Linux   4.19.115-3.ph3   containerd://1.3.4
tkg-guest-md-0-7b9ccb8bd7-x62rr   Ready    <none>   2d17h   v1.18.2+vmware.1   10.10.3.168   10.10.3.168   VMware Photon OS/Linux   4.19.115-3.ph3   containerd://1.3.4


# Tear down environment

In [7]:
#remove cf app from cluster
kapp delete -a cf -y

Target cluster 'https://10.10.3.159:6443' (nodes: tkg-guest-control-plane-7xnn9, 3+)

Changes

Namespace             Name                                                            Kind                          Conds.  Age  Op      Wait to  Rs  Ri  
(cluster)             adapters.config.istio.io                                        CustomResourceDefinition      2/2 t   30m  delete  delete   ok  -  
^                     aggregate-metacontroller-edit                                   ClusterRole                   -       29m  delete  delete   ok  -  
^                     aggregate-metacontroller-view                                   ClusterRole                   -       29m  delete  delete   ok  -  
^                     attributemanifests.config.istio.io                              CustomResourceDefinition      2/2 t   30m  delete  delete   ok  -  
^                     authorizationpolicies.security.istio.io                         CustomResourceDefinition      2/2 t   30m  delet

^                     istio-policy                                                    ClusterRole                   -       29m  delete  delete   ok  -  
^                     istio-policy-admin-role-binding-istio-system                    ClusterRoleBinding            -       29m  delete  delete   ok  -  
^                     istio-reader-istio-system                                       ClusterRole                   -       29m  delete  delete   ok  -  
^                     istio-reader-istio-system                                       ClusterRoleBinding            -       29m  delete  delete   ok  -  
^                     istio-sidecar-injector                                          MutatingWebhookConfiguration  -       29m  delete  delete   ok  -  
^                     istio-sidecar-injector-admin-role-binding-istio-system          ClusterRoleBinding            -       29m  delete  delete   ok  -  
^                     istio-sidecar-injector-istio-system                   

^                     cf-blobstore-minio                                              ConfigMap                     -       29m  delete  delete   ok  -  
^                     cf-blobstore-minio                                              Deployment                    2/2 t   29m  delete  delete   ok  -  
^                     cf-blobstore-minio                                              Endpoints                     -       29m  -       delete   ok  -  
^                     cf-blobstore-minio                                              PersistentVolumeClaim         -       29m  delete  delete   ok  -  
^                     cf-blobstore-minio                                              Secret                        -       29m  delete  delete   ok  -  
^                     cf-blobstore-minio                                              Service                       -       29m  delete  delete   ok  -  
^                     cf-blobstore-minio                                    

^                     cfroutesync                                                     Service                       -       29m  delete  delete   ok  -  
^                     cfroutesync-8999f4677                                           ReplicaSet                    -       29m  -       delete   ok  -  
^                     cfroutesync-8999f4677-z6tf2                                     Pod                           4/4 t   28m  -       delete   ok  -  
^                     cfroutesync-auth-metacontroller                                 AuthorizationPolicy           -       29m  delete  delete   ok  -  
^                     cfroutesync-auth-prometheus                                     AuthorizationPolicy           -       29m  delete  delete   ok  -  
^                     cfroutesync-config-ver-1                                        ConfigMap                     -       29m  delete  delete   ok  -  
^                     cloud-controller-ng-yaml-ver-1                        

^                     uaa-84b9d787cc                                                  ReplicaSet                    -       29m  -       delete   ok  -  
^                     uaa-84b9d787cc-wp28k                                            Pod                           4/4 t   28m  -       delete   ok  -  
^                     uaa-certs                                                       Secret                        -       29m  delete  delete   ok  -  
^                     uaa-config                                                      ConfigMap                     -       29m  delete  delete   ok  -  
^                     uaa-external-virtual-service                                    VirtualService                -       29m  delete  delete   ok  -  
cf-workloads          allow-app-ingress-from-ingressgateway                           NetworkPolicy                 -       29m  delete  delete   ok  -  
^                     app-registry-credentials                              

...

9:41:24AM: ok: delete rule/promhttp (config.istio.io/v1alpha2) namespace: istio-system
9:41:24AM: ongoing: delete replicaset/istio-galley-94b676666 (apps/v1) namespace: istio-system
9:41:24AM: ok: delete deployment/capi-kpack-watcher (apps/v1) namespace: cf-system
9:41:24AM: ok: delete clusterrole/aggregate-metacontroller-edit (rbac.authorization.k8s.io/v1) cluster
9:41:24AM: ok: delete clusterrolebinding/istio-pilot-istio-system (rbac.authorization.k8s.io/v1) cluster
9:41:24AM: ok: delete networkpolicy/pilot-network-policy (networking.k8s.io/v1) namespace: istio-system
9:41:24AM: ongoing: delete replicaset/capi-api-server-7c9979b866 (apps/v1) namespace: cf-system
9:41:24AM: ongoing: delete serviceaccount/default (v1) namespace: build-service
9:41:24AM: ok: delete service/istio-citadel (v1) namespace: istio-system
9:41:24AM: ok: delete secret/cfroutesync (v1) namespace: cf-system
9:41:24AM: ok: delete controllerrevision/istio-ingressgateway-5799c46547 (apps/v1) namespace: istio-system


9:41:24AM: ok: delete secret/log-cache-metrics (v1) namespace: cf-system
9:41:24AM: ok: delete secret/cc-kpack-registry-auth-secret (v1) namespace: cf-workloads-staging
9:41:24AM: ok: delete clusterrolebinding/kpack-watcher-binding (rbac.authorization.k8s.io/v1) cluster
9:41:24AM: ok: delete endpoints/istio-telemetry (v1) namespace: istio-system
9:41:24AM: ok: delete mutatingwebhookconfiguration/resource.webhook.kpack.pivotal.io (admissionregistration.k8s.io/v1beta1) cluster
9:41:24AM: ok: delete configmap/nginx-ver-1 (v1) namespace: cf-system
9:41:24AM: ongoing: delete pod/fluentd-hjq92 (v1) namespace: cf-system
9:41:24AM: ongoing: delete pod/istio-policy-7cbcb7f5c6-brf2x (v1) namespace: istio-system
9:41:24AM: ok: delete service/istio-galley (v1) namespace: istio-system
9:41:24AM: ok: delete instance/tcpbytesent (config.istio.io/v1alpha2) namespace: istio-system
9:41:24AM: ok: delete role/cf-workloads-privileged-app-role (rbac.authorization.k8s.io/v1) namespace: cf-workloads
9:41:24A

9:41:25AM: ok: delete endpoints/istio-policy (v1) namespace: istio-system
9:41:25AM: ongoing: delete pod/capi-kpack-watcher-7b5c8758d5-nj8lz (v1) namespace: cf-system
9:41:25AM: ok: delete replicaset/eirini-848f5c568 (apps/v1) namespace: cf-system
9:41:25AM: ok: delete clusterrole/istio-sidecar-injector-istio-system (rbac.authorization.k8s.io/v1) cluster
9:41:25AM: ok: delete serviceaccount/default (v1) namespace: kpack
9:41:25AM: ok: delete rolebinding/eirini-rolebinding (rbac.authorization.k8s.io/v1) namespace: cf-workloads
9:41:25AM: ok: delete serviceaccount/istio-citadel-service-account (v1) namespace: istio-system
9:41:25AM: ok: delete poddisruptionbudget/istio-galley (policy/v1beta1) namespace: istio-system
9:41:25AM: ok: delete rule/tcpkubeattrgenrulerule (config.istio.io/v1alpha2) namespace: istio-system
9:41:25AM: ok: delete replicaset/istio-telemetry-8c9887fd (apps/v1) namespace: istio-system
9:41:25AM: ok: delete secret/system-registry-credentials (v1) namespace: cf-workloa

9:41:59AM: ---- waiting on 1 changes [295/332 done] ----
9:42:06AM: ok: delete namespace/cf-blobstore (v1) cluster
9:42:06AM: ---- applying 36 changes [296/332 done] ----
9:42:06AM: delete customresourcedefinition/rbacconfigs.rbac.istio.io (apiextensions.k8s.io/v1beta1) cluster
9:42:06AM: delete customresourcedefinition/rules.config.istio.io (apiextensions.k8s.io/v1beta1) cluster
9:42:06AM: delete customresourcedefinition/envoyfilters.networking.istio.io (apiextensions.k8s.io/v1beta1) cluster
9:42:06AM: delete customresourcedefinition/virtualservices.networking.istio.io (apiextensions.k8s.io/v1beta1) cluster
9:42:06AM: delete customresourcedefinition/httpapispecbindings.config.istio.io (apiextensions.k8s.io/v1beta1) cluster
9:42:06AM: delete customresourcedefinition/quotaspecbindings.config.istio.io (apiextensions.k8s.io/v1beta1) cluster
9:42:06AM: delete customresourcedefinition/serviceroles.rbac.istio.io (apiextensions.k8s.io/v1beta1) cluster
9:42:06AM: delete customresourcedefinitio

9:42:10AM: ok: delete customresourcedefinition/builds.build.pivotal.io (apiextensions.k8s.io/v1beta1) cluster
9:42:10AM: ok: delete customresourcedefinition/sourceresolvers.build.pivotal.io (apiextensions.k8s.io/v1beta1) cluster
9:42:10AM: ok: delete customresourcedefinition/attributemanifests.config.istio.io (apiextensions.k8s.io/v1beta1) cluster
9:42:10AM: ok: delete customresourcedefinition/adapters.config.istio.io (apiextensions.k8s.io/v1beta1) cluster
9:42:10AM: ---- applying complete [332/332 done] ----
9:42:10AM: ---- waiting complete [332/332 done] ----

Succeeded
